[![Labellerr](https://storage.googleapis.com/labellerr-cdn/%200%20Labellerr%20template/notebook.webp)](https://www.labellerr.com)

# **Egg Cell Detection using YOLO**

---

[![labellerr](https://img.shields.io/badge/Labellerr-BLOG-black.svg)](https://www.labellerr.com/blog/<BLOG_NAME>)
[![Youtube](https://img.shields.io/badge/Labellerr-YouTube-b31b1b.svg)](https://www.youtube.com/@Labellerr)
[![Github](https://img.shields.io/badge/Labellerr-GitHub-green.svg)](https://github.com/Labellerr/Hands-On-Learning-in-Computer-Vision)

This notebook presents a **complete pipeline** for detecting and tracking egg cells from microscopic video footage using deep learning, specifically the **YOLO segmentation model**. Its workflow is designed for biologists, computer vision researchers, and ML engineers interested in automated analysis of cell videos.

## 1. Cloning YOLO Utilities Repository
**Command:**


In [1]:
!git clone https://github.com/Labellerr/yolo_finetune_utils.git

Cloning into 'yolo_finetune_utils'...


**Purpose:**  
This cell clones a specialized YOLO fine-tuning utility repository from GitHub. These scripts offer handy functions to help with annotation conversion and dataset setup.

---

## 2. Importing YOLO Conversion Utility
**Command:**


In [2]:
from yolo_finetune_utils.video_annotation.yolo_converter import convert_to_yolo_segmentation

**Purpose:**  
Imports a function that will later be used to convert custom video/book annotation data to the YOLO segmentation format required for training.

---

## 3. Converting JSON Annotations to YOLO Dataset
**Command:**


In [3]:
ANNOTATION_FILE = "annotations.json"
VIDEOS_DIRECTORY = "video"
OUTPUT_DATASET_DIR = "yolo_dataset"


convert_to_yolo_segmentation(
    annotation_path=ANNOTATION_FILE,
    videos_dir=VIDEOS_DIRECTORY,
    use_split=True,
    split_ratio=(0.9, 0.1, 0.0),
    output_dir=OUTPUT_DATASET_DIR
)

Using split ratio: Train=0.9, Val=0.1, Test=0.0
Creating output directory structure in: yolo_dataset
Loading JSON annotation file...
Found classes: ['Egg']
Aggregating annotations from JSON...


Processing files in JSON: 100%|██████████| 1/1 [00:00<00:00, 22.64it/s]



Extracting frames and saving to YOLO format...
Processing train set (267 frames)...


Exporting train set: 100%|██████████| 267/267 [00:08<00:00, 31.79it/s]


Processing val set (30 frames)...


Exporting val set: 100%|██████████| 30/30 [00:01<00:00, 22.86it/s]


No frames to process for the 'test' set.

Creating data.yaml file...

Conversion complete! ✨
Dataset saved to: d:\Professional\Projects\Egg_cell_detection_using_YOLO\yolo_dataset
YAML file saved to: d:\Professional\Projects\Egg_cell_detection_using_YOLO\yolo_dataset\data.yaml


**Purpose:**  
Transforms annotation data in JSON format into a YOLO-compatible dataset. It also splits the data (90% training, 10% validation, 0% test) and organizes it for training.

---

## 4. Emptying GPU Cache (PyTorch)
**Command:**


In [4]:
import torch
torch.cuda.empty_cache()

**Purpose:**  
Clears the GPU memory cache, freeing up RAM for future model training or inference runs.

---

## 5. Monitoring GPU Status
**Command:**


In [5]:
!nvidia-smi

Mon Nov 10 16:02:24 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 581.57                 Driver Version: 581.57         CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 5060 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   54C    P4             12W /   49W |     808MiB /   8151MiB |     12%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

**Purpose:**  
Displays key details about the current GPU hardware: memory usage, active processes, CUDA software version, and more.

---

## 6. Checking GPU Memory (PyTorch)
**Command:**


In [6]:
# Check GPU memory status
print(f"Allocated: {torch.cuda.memory_allocated(0)/1024**3:.2f} GB")
print(f"Cached: {torch.cuda.memory_reserved(0)/1024**3:.2f} GB")
print(f"Free: {torch.cuda.mem_get_info(0)[0]/1024**3:.2f} GB")

Allocated: 0.00 GB
Cached: 0.00 GB
Free: 6.78 GB


**Purpose:**  
Shows how much GPU memory is currently allocated, cached, and free, helping debug memory errors and optimize resource usage.

---

## 7. Importing Ultralytics YOLO Framework
**Command:**


In [7]:
from ultralytics import YOLO

**Purpose:**  
Imports the latest YOLO implementation from Ultralytics, enabling model loading, training, inference, and much more.

---

## 8. Training YOLO Segmentation Model
**Command:**


In [ ]:
# Load a model
model = YOLO("yolo11n-seg.pt")

# Train the model
results = model.train(
    data=r"d:\Professional\Projects\Egg_cell_detection_using_YOLO\yolo_dataset\data.yaml",    # Path to your dataset YAML file
    epochs=150,                        # Number of training epochs
    imgsz=640,                         # Image size
    batch=-1,                          # Batch size
    device=0,                          # GPU device (0 for first GPU, 'cpu' for CPU)
    workers=4,                         # Number of dataloader workers
)


**Purpose:**  
Loads a YOLO-v11 segmentation model and trains on your custom dataset for 150 epochs.  
Key details: image size 640px, GPU device 0, using the dataset you organized earlier.

---

## 9. Running Inference & Extracting Results
**Command:**


In [8]:
model = YOLO(r"runs\segment\train\weights\best.pt")

# Run inference on video
results = model.track(
    source=r"video\embryo.mp4",
    save=True,
    conf=0.25,
    iou=0.7,
    show=True,
    show_labels=False
)


WARNING 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/297) d:\Professional\Projects\Egg_cell_detection_using_YOLO\video\embryo.mp4: 480x640 3 Eggs, 59.6ms
video 1/1 (frame 2/297) d:\Professional\Projects\Egg_cell_detection_using_YOLO\video\embryo.mp4: 480x640 3 Eggs, 49.6ms
video 1/1 (frame 3/297) d:\Professional\Projects\Egg_cell_detection_using_YOLO\video\embryo.mp4: 480x640 3 Eggs, 60.6ms
video 1/1 (frame 4/297) d:\Professional\Projects\Egg_cell_detection_using_YOLO\video\embryo.mp4: 48

**Purpose:**  
Runs inference with streaming output (frame-by-frame) so as to avoid RAM issues.  
Collects bounding boxes, segmentation masks, and class probabilities for downstream processing.

---

## 10. Egg Cell Tracker Class Definition
**Command:**


In [16]:
import cv2
import numpy as np
from ultralytics import YOLO
from collections import defaultdict
import colorsys


class EggCellTracker:
    """
    A class for tracking and visualizing microscopic egg cells using YOLO segmentation.
    
    Features:
    - Object tracking across frames
    - Segmentation mask visualization with unique colors
    - Distance measurement between nearby objects
    """
    
    def __init__(self, model_path, distance_threshold=300):
        """
        Initialize the tracker.
        
        Args:
            model_path (str): Path to the YOLO model
            distance_threshold (int): Maximum distance to draw lines between objects (default: 300)
        """
        self.model = YOLO(model_path)
        self.distance_threshold = distance_threshold
        self.track_colors = {}
        self.track_history = defaultdict(list)
        
    def generate_color(self, track_id):
        """Generate a unique color for each track ID."""
        if track_id not in self.track_colors:
            # Generate distinct colors using HSV color space
            hue = (track_id * 0.618033988749895) % 1.0  # Golden ratio for better distribution
            rgb = colorsys.hsv_to_rgb(hue, 0.8, 0.95)
            self.track_colors[track_id] = tuple(int(c * 255) for c in rgb)
        return self.track_colors[track_id]
    
    def calculate_centroid(self, mask):
        """Calculate the centroid of a segmentation mask."""
        moments = cv2.moments(mask)
        if moments["m00"] != 0:
            cx = int(moments["m10"] / moments["m00"])
            cy = int(moments["m01"] / moments["m00"])
            return (cx, cy)
        return None
    
    def calculate_distance(self, point1, point2):
        """Calculate Euclidean distance between two points."""
        return np.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)
    
    def draw_segmentation_mask(self, frame, mask, color, alpha=0.5):
        """Draw a colored segmentation mask on the frame."""
        colored_mask = np.zeros_like(frame, dtype=np.uint8)
        colored_mask[mask > 0] = color
        
        # Blend the mask with the original frame
        frame = cv2.addWeighted(frame, 1, colored_mask, alpha, 0)
        
        # Draw contour for better visibility
        contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(frame, contours, -1, color, 2)
        
        return frame
    
    def draw_distance_lines(self, frame, centroids, colors):
        """Draw lines between nearby objects with distance labels."""
        centroid_list = list(centroids.items())
        lines_drawn = 0
        
        for i in range(len(centroid_list)):
            for j in range(i + 1, len(centroid_list)):
                track_id1, center1 = centroid_list[i]
                track_id2, center2 = centroid_list[j]
                
                distance = self.calculate_distance(center1, center2)
                
                if distance <= self.distance_threshold:
                    # Draw line between centroids with thicker line
                    cv2.line(frame, center1, center2, (0, 255, 255), 3)  # Yellow line, thicker
                    
                    # Calculate midpoint for distance label
                    mid_x = (center1[0] + center2[0]) // 2
                    mid_y = (center1[1] + center2[1]) // 2
                    
                    # Draw distance text with background
                    distance_text = f"{distance:.1f}px"
                    (text_width, text_height), baseline = cv2.getTextSize(
                        distance_text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2
                    )
                    
                    # Draw background rectangle
                    cv2.rectangle(frame,
                                (mid_x - 2, mid_y - text_height - 2),
                                (mid_x + text_width + 2, mid_y + baseline + 2),
                                (0, 0, 0), -1)
                    
                    # Draw text
                    cv2.putText(frame, distance_text, (mid_x, mid_y),
                              cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
                    
                    lines_drawn += 1
        
        return frame, lines_drawn
    
    def process_video(self, video_path, output_path=None, show_preview=True, draw_distance_lines=True):
        """
        Process a video file and track egg cells.
        
        Args:
            video_path (str): Path to input video
            output_path (str): Path to save output video (optional)
            show_preview (bool): Whether to show live preview
        """
        cap = cv2.VideoCapture(video_path)
        
        # Get video properties
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        
        # Initialize video writer if output path is provided
        writer = None
        if output_path:
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
        
        frame_count = 0
        
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            frame_count += 1
            
            # Run tracking
            results = self.model.track(frame, persist=True, verbose=False)
            
            # Create a copy for annotation
            annotated_frame = frame.copy()
            
            # Store centroids for distance calculation
            current_centroids = {}
            current_colors = {}
            
            if results[0].masks is not None and results[0].boxes.id is not None:
                # Get masks and track IDs
                masks = results[0].masks.data.cpu().numpy()
                track_ids = results[0].boxes.id.cpu().numpy().astype(int)
                
                # Process each detection
                for mask, track_id in zip(masks, track_ids):
                    # Resize mask to frame size
                    mask_resized = cv2.resize(mask, (width, height))
                    mask_binary = (mask_resized > 0.5).astype(np.uint8)
                    
                    # Generate color for this track
                    color = self.generate_color(track_id)
                    current_colors[track_id] = color
                    
                    # Draw segmentation mask
                    annotated_frame = self.draw_segmentation_mask(
                        annotated_frame, mask_binary, color
                    )
                    
                    # Calculate and store centroid
                    centroid = self.calculate_centroid(mask_binary)
                    if centroid:
                        current_centroids[track_id] = centroid
                        
                        # Draw centroid point
                        cv2.circle(annotated_frame, centroid, 5, color, -1)
                        cv2.circle(annotated_frame, centroid, 5, (255, 255, 255), 2)
                        
                        # Draw track ID with background
                        label = f"ID:{track_id}"
                        label_x = centroid[0] + 10
                        label_y = centroid[1] - 10
                        
                        # Get text size for background rectangle
                        (text_width, text_height), baseline = cv2.getTextSize(
                            label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2
                        )
                        
                        # Draw background rectangle with track color
                        cv2.rectangle(annotated_frame, 
                                    (label_x - 2, label_y - text_height - 2),
                                    (label_x + text_width + 2, label_y + baseline + 2),
                                    color, -1)
                        
                        # Draw white text on colored background
                        cv2.putText(annotated_frame, label, 
                                  (label_x, label_y),
                                  cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            
            # Draw distance lines between nearby objects
            if draw_distance_lines:
                lines_drawn = 0
                if len(current_centroids) > 1:
                    annotated_frame, lines_drawn = self.draw_distance_lines(
                        annotated_frame, current_centroids, current_colors
                    )
            
            # Add object counter and lines info
            cv2.putText(annotated_frame, f"Egg Cells: {len(current_centroids)}", (10, 30),
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            
            # Write frame
            if writer:
                writer.write(annotated_frame)
            
            # Show preview
            if show_preview:
                cv2.imshow('Egg Cell Tracking', annotated_frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
        
        # Cleanup
        cap.release()
        if writer:
            writer.release()
        if show_preview:
            cv2.destroyAllWindows()
        
        print(f"Processing complete! Total frames: {frame_count}")
        print(f"Total unique tracks: {len(self.track_colors)}")



**Purpose:**  
Defines the main class for post-processing YOLO outputs: tracks segmented egg cells across frames, annotates videos, manages object IDs, and calculates movement or proximity between detected cells.

---

## 11. Using the Egg Cell Tracker (Example)
**Command:**


In [ ]:
# Example usage
if __name__ == "__main__":
    # Initialize tracker
    tracker = EggCellTracker(
        model_path=r"D:\Professional\Projects\Egg_cell_detection_using_YOLO\runs\segment\train\weights\best.pt",
        distance_threshold=500  # Adjust based on your needs
    )
    
    # Process video
    tracker.process_video(
        video_path=r"D:\Professional\Projects\Egg_cell_detection_using_YOLO\video\embryo.mp4",
        output_path="egg_cell_detection.mp4",
        show_preview=True,
        draw_distance_lines=False
    )

Processing complete! Total frames: 297
Total unique tracks: 3


**Purpose:**  
Demonstrates how to initialize the custom tracker with trained weights, set cell-tracking distance, analyze a raw micro-video, and produce a fully annotated output file.

---

*This notebook sets up, trains, and analyzes fertile egg cells using advanced YOLO segmentation and custom video processing classes. Each section includes a code fragment and its intended role within the workflow.*
